In [1]:
%%capture
!pip install fairlearn

In [3]:
import numpy as np
from IPython.display import display, Markdown, Latex
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.metrics import MetricFrame
import fairlearn.datasets as fdata
from fairlearn.metrics import (demographic_parity_difference, demographic_parity_ratio,
                               selection_rate_difference, false_negative_rate_difference,
                               false_positive_rate_difference, equalized_odds_ratio,
                               false_negative_rate, false_positive_rate)

## NOTE: If the top two cells give you errors:
### 1. Open `Runtime` in the top bar
### 2. Press `Restart session`
### 3. Try running the cells again
#### (This is not the same thing as reloading the page!)

(if this still doesn't work, post a **private** message on Piazza or come to hours)

# Load and preprocess the data

In [4]:
#@markdown Load and read about the dataset.
# get datast from fairlearn and show description
dataset = fdata.fetch_diabetes_hospital()

display(Markdown(dataset.DESCR))

# save dataframe and features
x_raw = dataset.data
# y_raw = np.array(dataset.target)
feature_names = dataset.feature_names

The "Diabetes 130-Hospitals" dataset represents 10 years of clinical care at 130 U.S. hospitals and delivery networks, collected from 1999 to 2008. Each record represents the hospital admission record for a patient diagnosed with diabetes whose stay lasted between one to fourteen days. The features describing each encounter include demographics, diagnoses, diabetic medications, number of visits in the year preceding the encounter, and payer information, as well as whether the patient was readmitted after release, and whether the readmission occurred within 30 days of the release.

The original "Diabetes 130-Hospitals" dataset was collected by Beata Strack, Jonathan P. DeShazo, Chris Gennings, Juan L. Olmo, Sebastian Ventura, Krzysztof J. Cios, and John N. Clore in 2014.

This version of the dataset was derived by the Fairlearn team for the SciPy 2021 tutorial "Fairness in AI Systems: From social context to practice using Fairlearn". In this version, the target variable "readmitted" is binarized into whether the patient was re-admitted within thirty days. The full dataset pre-processing script can be found on GitHub: https://github.com/fairlearn/talks/blob/main/2021_scipy_tutorial/preprocess.py

Downloaded from openml.org.

You can read more about the dataset [here](https://fairlearn.org/main/user_guide/datasets/diabetes_hospital_data.html). In this description, we see that two features, `readmitted` and `readmit_binary`, are other representations of the same outcome, so we drop them from the set of predictors.

In [4]:
#@markdown Down sample to make runtimes reasonable
x_raw = x_raw.sample(frac=0.1, random_state=123)

In [5]:
y_raw = x_raw['readmit_binary']
x_raw = x_raw.drop(columns=['readmitted', 'readmit_binary'])
feature_names = feature_names[:-2]

In [6]:
#@markdown Look at the first few rows of the data.
x_raw.head()

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,max_glu_serum,A1Cresult,insulin,change,diabetesMed,medicare,medicaid,had_emergency,had_inpatient_days,had_outpatient_days
65884,Caucasian,Male,'Over 60 years','Discharged to Home',Emergency,3,Missing,49,6,27,...,NaN,NaN,Up,Ch,Yes,False,False,False,True,False
86278,Caucasian,Female,'Over 60 years',Other,Referral,5,Missing,57,0,21,...,NaN,NaN,No,Ch,Yes,True,False,False,True,False
30000,Caucasian,Male,'30-60 years','Discharged to Home',Other,4,Other,37,2,9,...,NaN,NaN,No,No,No,False,False,False,False,False
51185,Caucasian,Male,'30-60 years',Other,Other,9,InternalMedicine,61,6,27,...,NaN,>8,No,Ch,Yes,False,False,True,False,True
53902,Caucasian,Female,'30-60 years','Discharged to Home',Referral,7,Other,23,0,13,...,NaN,NaN,No,No,Yes,True,False,True,True,False


In [7]:
# check how many 'Unknown/Invalid' values there are for gender
(x_raw.gender == 'Unknown/Invalid').sum()

0

In [8]:
# drop these 3 rows
print(x_raw.shape)
rows_to_keep = x_raw.gender != 'Unknown/Invalid'
x_raw = x_raw[rows_to_keep]
y_raw = y_raw[rows_to_keep]
print(x_raw.shape)

(10177, 22)
(10177, 22)


In [9]:
x_raw.dtypes

race                          object
gender                        object
age                           object
discharge_disposition_id      object
admission_source_id           object
time_in_hospital               int64
medical_specialty             object
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
primary_diagnosis             object
number_diagnoses               int64
max_glu_serum                 object
A1Cresult                     object
insulin                       object
change                        object
diabetesMed                   object
medicare                    category
medicaid                    category
had_emergency               category
had_inpatient_days          category
had_outpatient_days         category
dtype: object

In [12]:
unique_feature_values = x_raw.apply(lambda col: col.unique())
unique_feature_values

race                        [Caucasian, AfricanAmerican, Unknown, Other, A...
gender                                                         [Male, Female]
age                         ['Over 60 years', '30-60 years', '30 years or ...
discharge_disposition_id                        ['Discharged to Home', Other]
admission_source_id                              [Emergency, Referral, Other]
time_in_hospital              [3, 5, 4, 9, 7, 8, 11, 2, 1, 6, 12, 13, 14, 10]
medical_specialty           [Missing, Other, InternalMedicine, Emergency/T...
num_lab_procedures          [49, 57, 37, 61, 23, 65, 47, 35, 51, 22, 46, 4...
num_procedures                                          [6, 0, 2, 1, 3, 4, 5]
num_medications             [27, 21, 9, 13, 40, 17, 34, 10, 19, 11, 6, 14,...
primary_diagnosis           [Other, Diabetes, 'Respiratory Issues', 'Muscu...
number_diagnoses                  [9, 5, 7, 4, 6, 8, 3, 2, 1, 16, 15, 13, 11]
max_glu_serum                                         [nan, Norm

In [13]:
#@markdown
binary_features = unique_feature_values.index[[len(x) == 2 for x in unique_feature_values]].values
print(f'Binary features: {binary_features}')
categorical_features = unique_feature_values.index[[len(x) > 2 and isinstance(x[0], str) for x in unique_feature_values]].values
print(f'Categorical features: {categorical_features}')

Binary features: ['gender' 'discharge_disposition_id' 'change' 'diabetesMed' 'medicare'
 'medicaid' 'had_emergency' 'had_inpatient_days' 'had_outpatient_days']
Categorical features: ['race' 'age' 'admission_source_id' 'medical_specialty'
 'primary_diagnosis' 'insulin']


In [14]:
#@markdown
for col_name in feature_names:
    if col_name in categorical_features:
        x_raw[col_name] = x_raw[col_name].astype('category')
    elif col_name in binary_features:  # redundant for clarity
        # turn into int column
        integer_col = (x_raw[col_name] == unique_feature_values[col_name][0]).astype(int)
        new_name = f'{col_name}_{unique_feature_values[col_name][0]}'
        x_raw[new_name] = integer_col
        x_raw.drop(columns=[col_name], inplace=True)

In [16]:
#@markdown
%%capture
x_raw.apply(lambda col: col.unique())


In [17]:
x_raw.dtypes

race                                             category
age                                              category
admission_source_id                              category
time_in_hospital                                    int64
medical_specialty                                category
num_lab_procedures                                  int64
num_procedures                                      int64
num_medications                                     int64
primary_diagnosis                                category
number_diagnoses                                    int64
max_glu_serum                                      object
A1Cresult                                          object
insulin                                          category
gender_Male                                         int64
discharge_disposition_id_'Discharged to Home'       int64
change_Ch                                           int64
diabetesMed_Yes                                     int64
medicare_False

In [18]:
#@markdown One-hot encode categorical features
x_numeric = pd.get_dummies(x_raw)
display(x_numeric.head())

# get one-hot and numeric column names
numeric_cols = x_numeric.dtypes.index[x_numeric.dtypes == 'float64'].values
one_hot_cols = x_numeric.dtypes.index[x_numeric.dtypes != 'float64'].values

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,gender_Male,discharge_disposition_id_'Discharged to Home',change_Ch,diabetesMed_Yes,medicare_False,...,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_Norm,insulin_Down,insulin_No,insulin_Steady,insulin_Up
65884,3,49,6,27,9,1,1,1,1,1,...,False,False,False,False,False,False,False,False,False,True
86278,5,57,0,21,9,0,0,1,1,0,...,False,False,False,False,False,False,False,True,False,False
30000,4,37,2,9,5,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,False
51185,9,61,6,27,9,1,0,1,1,1,...,False,False,False,False,True,False,False,True,False,False
53902,7,23,0,13,7,0,1,0,1,0,...,False,False,False,False,False,False,False,True,False,False


In [19]:
#@markdown ## TODO: Split train, test, validation data 70 train/20 test/10 val


In [ ]:
#@markdown ## TODO: Initialize Fairlearn Metric Dictrs and sensitive features

# Problem 2, Part (a)
### **Train a baseline Random Forest (RF) (sklearn RandomForestClassifier) model and report metrics**

### Train a random forest model - Baseline

### Report metrics

In [ ]:
#@markdown ### Calculate metrics with Fairlearn MetricFrame


In [ ]:
#@markdown Print Results

In [ ]:
#@markdown # Problem 2, Part (b)
#@markdown ### **Hyperparameter tuning of baseline RF model**
def tune_random_forest(x_train, y_train, x_val, y_val, max_depth, n_estimators, verbose=False, weights=None):
    '''Tunes logistic regression models over the hyperparameters max depth and n_estimators
       to maximize the AUC'''
    pass

In [ ]:
#@markdown Tuning loop across different seeds


In [ ]:
#@markdown ### Boxplot tuned/untuned comparison for each metric

# Problem 2, Part (c)
### **Fit Fairlean Adversarial Debiaser**

Experiment with the alpha parameter - which value of alpha produces the fairest and most accurate classifier? Does such a value exist?

In [ ]:
#@markdown Fit the AdversarialFairnessClassifier here.
#@markdown Use these hyperparameters, while varying the `alpha` parameter:
#@markdown - backend='tensorflow',
#@markdown - predictor_model=[128,64,32,16,8],
#@markdown - adversary_model=[32,16,8],
#@markdown - learning_rate=0.001,
#@markdown - epochs=3,
#@markdown - batch_size=16,
#@markdown - constraints='demographic_parity',
#@markdown - random_state=seed,
#@markdown - shuffle=True

In [ ]:
#@markdown ### Plot all our metrics as line plots while varying alpha

# Problem 2, Part (d)
### **Threshold Optimizer Post-processing intervention**



In [ ]:
#@markdown Fit the ThresholdOptimizer model here